In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit

qc = QuantumCircuit(3, 3)
qc.h([0,1,2])

qc.measure([0,1,2], [0,1,2])
qc.draw()

┌───┐┌─┐      
q_0: ┤ H ├┤M├──────
     ├───┤└╥┘┌─┐   
q_1: ┤ H ├─╫─┤M├───
     ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2

In [4]:
# Get the API token in https://quantum-computing.ibm.com/
#QiskitRuntimeService.save_account(channel="ibm_quantum", token="TOKEN")

service = QiskitRuntimeService()
backend = service.least_busy()
print (backend)

<IBMBackend('ibmq_qasm_simulator')>


save_account stores the API token to disk, hence it must be done only once after rotating the token. See https://github.com/Qiskit/qiskit-ibmq-provider for more details.

In [5]:
with Sampler(circuits=qc, service=service, options={ "backend": "ibmq_quito" }) as sampler:
    # pass indices of circuits
    result = sampler(circuits=[0], shots=1024)
    print(result)

SamplerResult(quasi_dists=[{'000': 0.1435546875, '001': 0.12890625, '010': 0.119140625, '011': 0.1318359375, '100': 0.142578125, '101': 0.123046875, '110': 0.1171875, '111': 0.09375}], metadata=[{'header_metadata': {}, 'shots': 1024}])
